In [22]:
import pandas as pd
import ast

In [23]:
movies = pd.read_csv('/kaggle/input/tmdb-5k/tmdb_5000_movies.csv')
credits = pd.read_csv('/kaggle/input/tmdb-5k/tmdb_5000_credits.csv')

In [24]:
credits.rename(columns={'movie_id': 'id'}, inplace=True)
movies = movies.merge(credits[['id', 'cast', 'crew']], on='id')

In [25]:
movies = movies[['id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.dropna(inplace=True)

In [26]:
def parse_names(obj):
    return [i['name'] for i in ast.literal_eval(obj)]

movies['genres'] = movies['genres'].apply(parse_names)
movies['keywords'] = movies['keywords'].apply(parse_names)

In [27]:
def top3_cast(obj):
    cast = ast.literal_eval(obj)
    return [i['name'] for i in cast[:3]]

movies['cast'] = movies['cast'].apply(top3_cast)

In [28]:
def get_director(obj):
    for i in ast.literal_eval(obj):
        if i.get('job') == 'Director':
            return [i['name']]
    return []

movies['crew'] = movies['crew'].apply(get_director)

In [29]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [30]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [31]:
new_df = movies[['id', 'title', 'tags']].copy()
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x).lower())
new_df['title'] = new_df['title'].apply(lambda x: x.lower())

In [32]:
new_df.to_csv('final_movie_data.csv', index=False)
print("Saved as final_movie_data.csv")

Saved as final_movie_data.csv
